In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size = 2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size = 2000)
lats_lngs = zip(lats, lngs)
lats_lngs

In [3]:
coordinates = list(lats_lngs)
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

768

In [4]:
cities[0:5]

['ushuaia', 'honiara', 'rikitea', 'aykhal', 'busselton']

In [5]:
url = 'http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=' + weather_api_key 
print(url)

http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=0c95010018f254d913dc916cf027dded


In [6]:
r = requests.get(url + "&q=" + cities[13].replace(" ","+") ).json()

In [7]:
r

{'coord': {'lon': -22.4338, 'lat': 63.8424},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 55.87,
  'feels_like': 55,
  'temp_min': 55.35,
  'temp_max': 55.87,
  'pressure': 1018,
  'humidity': 82},
 'visibility': 10000,
 'wind': {'speed': 8.05, 'deg': 160},
 'clouds': {'all': 75},
 'dt': 1658509190,
 'sys': {'type': 1,
  'id': 90,
  'country': 'IS',
  'sunrise': 1658462900,
  'sunset': 1658531020},
 'timezone': 0,
 'id': 3416888,
 'name': 'Grindavik',
 'cod': 200}

In [8]:
city_data = []
skip_count = 0
# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        skip_count += 1
        continue
# print api summary notes.
print('{} cities skipped.'.format(skip_count)) 
print('data for {} cities acquired '.format(len(city_data)))

61 cities skipped.
data for 707 cities acquired 


In [9]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ushuaia,AR,-54.8000,-68.3000,35.26,93,100,32.21,sleet
1,Honiara,SB,-9.4333,159.9500,76.14,85,39,3.02,scattered clouds
2,Rikitea,PF,-23.1203,-134.9692,72.61,75,99,7.70,overcast clouds
3,Aykhal,RU,66.0000,111.5000,53.19,84,64,2.39,broken clouds
4,Busselton,AU,-33.6500,115.3333,58.93,81,29,17.07,scattered clouds


In [10]:
city_data_df.to_csv('WeatherPy_Database.csv',index_label='City_ID')